The Python SDK for the Gemini API is contained in the google-generativeai package. Install the dependency using pip:

In [ ]:
!pip install -q -U google-generativeai

Set up API Key

In [ ]:
import os
os.environ["API_KEY"] = ""


In [ ]:
import google.generativeai as genai

genai.configure(api_key=os.environ["API_KEY"])

In [ ]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from json import loads,dumps
import time
from glob import glob

In [ ]:
import typing_extensions as typing

### here we are defining a JSON schema, because we want our output in a specific format, feel free to change this
class LLM_Output(typing.TypedDict):
    label: str
    language: list[str]
    response: str

Initializing model (need to do re-initialize everytime you change API Key)

In [ ]:
model_config = {
  "temperature": 0,
}

model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction="You are an expert content moderator.", generation_config=model_config)




In [ ]:
# Create the prompt.
prompt = """
    A piece of content is child-directed if it meets any of these criteria:

    Designed for Children:

    The content is explicitly created with children as the intended audience, such as educational videos for kids, children’s games, or storytelling aimed at young viewers.
    Child-Appealing Elements:

    The content includes features commonly enjoyed by children, such as:
    Animated characters, playful animals, family-friendly heroes, or simple adventures.
    Light-hearted portrayals of themes like mischief or conflict, but without mature visuals, strong language, or adult humor.
    Exclusions:

    Songs: Only label as child-directed if the song is a nursery rhyme or poem that is specifically created for children.
    Recipe, DIY, or Instructional Videos: Do not label as child-directed unless clearly simplified for children, like “Easy Cooking for Kids.”

    These are not child directed:
    videos with religious content
    videos with romantic themes including adults
    songs that are not nursery rhymes or poems


    Instructions
    Labeling:
    If the video meets the child-directed criteria label it as "true".
    If it does not meet these criteria label it as "false".

    Indicate the spoken language if any.
    Provide a brief justification explaining why the video is considered child-directed or not.

    Format the output in JSON.
  """

Run for sample of videos

In [ ]:
video_ids = [

    ] #list of video ids
redo_ids = [

] #video ids for which there was an error

print(len(video_ids))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
for i,v in enumerate(video_ids):

  ### change this folder path to the child-directed videos you are using
  ## e.g. folder_path = f"/content/drive/MyDrive/Safe Online Global/video-ads/{v}"

  folder_path =  f"/content/drive/MyDrive/non-madeForKids/{v}"

  if not os.path.exists(folder_path):
    print("Could not find folder for id:",v)
  else:

      video_mp4 = glob(f"{folder_path}/*.mp4") ## this line of code finds all the files in the folder path that have .mp4 extension
      audio_mp3 = glob(f"{folder_path}/*.mp3") ## this line of code finds all the files in the folder path that have .mp3 extension

      if len(video_mp4) == 0 or len(audio_mp3)==0:
        print("Could not find video/audio for id:",v)
      else:
        path_to_video = video_mp4[0] ## since we only have 1 video and 1 audio, just taking whatever is at the first index
        path_to_audio = audio_mp3[0]

        for attempt in range(5):
          try:
            ## upload video
            print(f"Uploading file...")
            video_file = genai.upload_file(path=path_to_video)
            print(f"Completed upload: {video_file.uri}")
            break

          except Exception as e:
              print(f"Upload video failed with error {e}. Retrying in {5} seconds...")
              time.sleep(5)

        for attempt in range(5):
          try:
            ## upload audio
            print(f"Uploading file...")
            audio_file = genai.upload_file(path=path_to_audio)
            print(f"Completed upload: {audio_file.uri}")
            break

          except Exception as e:
              print(f"Upload audio failed with error {e}. Retrying in {5} seconds...")
              time.sleep(5)


        ## check if video has uploaded - wait for it to upload
        while video_file.state.name == "PROCESSING":
          print('.', end='')
          time.sleep(10)
          video_file = genai.get_file(video_file.name)

        if video_file.state.name == "FAILED":
          raise ValueError(video_file.state.name)

        ################################################# make inference #################################################
        print("Making LLM inference request...")


        ## we pass in the audio file, video file and the prompt to the model
        ## we also change some settings by adjusting the configuration
        ## 1) we want output in JSON so we specify: response_mime_type="application/json"
        ## 2) we want the output in a specific JSON format/schema (so we know exactly what the keys are so we specific: response_schema=LLM_Output
        ## 3) because we want to generate descriptions/justifications of why a certain video is inappropriate, we removed the safety settings/put the threshold to NONE otherwise the LLM refrains from outputting content that is harmful

        response = model.generate_content([audio_file,video_file, prompt],
                                    generation_config=genai.GenerationConfig(
                                        response_mime_type="application/json", response_schema=LLM_Output),

                                    safety_settings={
          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
      })


        # print(response.text)

        ########################### save each output as a json file #################################################

        ### path where we will save, change this to whicherver folder you want to save outputs in
        ## e.g. path_to_save_json = f'/content/drive/MyDrive/Safe Online Global/gemini-trial-output/few-shot/{v}.json'
        path_to_save_json =  f'/content/drive/MyDrive/non-madeForKids_results/{v}.json'

        try:
          with open(path_to_save_json, 'w') as f:
            f.write(response.text)
        except:
          print("Could not save file for id:",v)
          redo_ids.append(v)

        ## Completed message
        print("Completed for video number:",i)

        ## wait a bit to avoid exceeding rate
        time.sleep(12)







In [ ]:
import os
import json
import csv

# Folder path containing JSON files
folder_path = f'/content/drive/MyDrive/non-madeForKids_results/'

# CSV file to store results
output_csv =  f'/content/drive/MyDrive/non-madeForKids_total.csv'

os.makedirs(os.path.dirname(output_csv), exist_ok=True)

# Open the CSV file for writing
with open(output_csv, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['ID', 'Label', 'Language'])  # Write the header

    # Loop through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            # Extract ID from the filename (remove the .json extension)
            id_value = os.path.splitext(filename)[0]

            # Construct full file path
            file_path = os.path.join(folder_path, filename)
            print(file_path)
            # Open and read the JSON file
            with open(file_path, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)

                # Extract 'Label' from the JSON data
                label_value = data.get('label', None)

                language_value = data.get('language', None)

                # Write the ID and Label to the CSV file if Label exists
                if label_value is not None:
                    writer.writerow([id_value, label_value, language_value])

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

data = pd.read_csv(f'/content/drive/MyDrive/total.csv'
)

# Extract labels and ground truth values
y_true = data['Ground Truth']
y_pred = data['Label']

# Generate and display classification report
report = classification_report(y_true, y_pred, output_dict=True)
print(classification_report(y_true, y_pred))
report_df = pd.DataFrame(report).transpose()

# Plot heatmap for confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=report_df.index[:-3], yticklabels=report_df.index[:-3])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix Heatmap')
plt.show()